<a href="https://colab.research.google.com/github/m10k1/anthropic-cookbook/blob/main/evaluator_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install anthropic

In [10]:
from google.colab import drive
mount_path = '/content/drive'
drive.mount(mount_path)

import os
import sys
sys.path.append(os.path.join(mount_path,'MyDrive/python'))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
from google.colab import userdata
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')

In [12]:
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Callable
from util import llm_call, extract_xml

# Evaluator-Optimizer ワークフロー

このワークフローでは、1つのLLMコールが応答を生成し、別のLLMコールが評価とフィードバックをループで提供する。

## このワークフローを使用する場合

このワークフローは、以下のような場合に特に効果的です：

* 明確な評価基準  
* 反復的改良による価値


このワークフローが適用に向いている２つの特長

* フィードバックが提供されると、LLMの回答が明らかに改善される。
* LLM自身が有意義なフィードバックを提供できる

![image](https://storage.googleapis.com/zenn-user-upload/b83cbd2ea240-20250110.png)

In [13]:
from util import llm_call, extract_xml

生成、フィードバックをもとにソリューションを改善


In [14]:
def generate(prompt: str, task: str, context: str = "") -> tuple[str, str]:
    """Generate and improve a solution based on feedback."""

    # プロンプト、コンテキスト、タスクの３つの要素
    full_prompt = f"{prompt}\n{context}\nTask: {task}" if context else f"{prompt}\nTask: {task}"

    #生成結果はthoughtsとresponseが含まれる
    response = llm_call(full_prompt)
    thoughts = extract_xml(response, "thoughts")
    result = extract_xml(response, "response")

    print("\n=== GENERATION START ===")
    print(f"Thoughts:\n{thoughts}\n")
    print(f"Generated:\n{result}")
    print("=== GENERATION END ===\n")

    return thoughts, result

評価を行う

ソリューションが要件に合致するか評価する


In [15]:

def evaluate(prompt: str, content: str, task: str) -> tuple[str, str]:
    """Evaluate if a solution meets requirements."""
    full_prompt = f"{prompt}\nOriginal task: {task}\nContent to evaluate: {content}"
    response = llm_call(full_prompt)
    evaluation = extract_xml(response, "evaluation")
    feedback = extract_xml(response, "feedback")

    print("=== EVALUATION START ===")
    print(f"Status: {evaluation}")
    print(f"Feedback: {feedback}")
    print("=== EVALUATION END ===\n")

    return evaluation, feedback

ループ処理

要件が満たされるまで、生成と評価を続ける。

In [16]:
def loop(task: str, evaluator_prompt: str, generator_prompt: str) -> tuple[str, list[dict]]:
    """Keep generating and evaluating until requirements are met."""
    memory = []
    chain_of_thought = []

    # 最初の生成
    thoughts, result = generate(generator_prompt, task)

    # 結果をメモリーに保存
    memory.append(result)

    chain_of_thought.append({"thoughts": thoughts, "result": result})

    # ループ
    while True:
        evaluation, feedback = evaluate(evaluator_prompt, result, task)

        # 要件を満たした場合にループ終了
        if evaluation == "PASS":
            return result, chain_of_thought

        context = "\n".join([
            "Previous attempts:",
            *[f"- {m}" for m in memory],
            f"\nFeedback: {feedback}"
        ])

        # 再生成
        thoughts, result = generate(generator_prompt, task, context)
        memory.append(result)
        chain_of_thought.append({"thoughts": thoughts, "result": result})

## 評価プロンプトの内容

```
以下のコード実装を評価しなさい
1. コードの正確性
2. 時間計算量
3. コーディングスタイルとベストプラクティス

あなたは評価のみを行います。そしてタスクを解こうとはしないでください。
もしすべての基準を満たし、それ以上の改善案が無い場合のみ「PASS」と出力してください。
評価は以下の形式で簡潔に出力してください

<evaluation>PASS, NEEDS_IMPROVEMENT, or FAIL</evaluation>
<feedback>
改善が必要な点とその理由
</feedback>


```


In [17]:
evaluator_prompt = """
Evaluate this following code implementation for:
1. code correctness
2. time complexity
3. style and best practices

You should be evaluating only and not attemping to solve the task.
Only output "PASS" if all criteria are met and you have no further suggestions for improvements.
Output your evaluation concisely in the following format.

<evaluation>PASS, NEEDS_IMPROVEMENT, or FAIL</evaluation>
<feedback>
What needs improvement and why.
</feedback>
"""

### 生成プロンプトの中身

```
あなたの目標はタスクを完了させることです。タスクは<ユーザーの入力>に基づいています。
もし、あなたの前の世代からのフィードバックがあれば、あなたの解決策を改善するためにそれを反映させます。

解答は以下の形式で簡潔に出力してください：

<thoughts>
[タスクとフィードバックに対するあなたの理解と、どのように改善する計画か？］  
</thoughts>  

<response>  
[ここにあなたのコードを実装してください］  
</response>
```


In [18]:
generator_prompt = """
Your goal is to complete the task based on <user input>. If there are feedback
from your previous generations, you should reflect on them to improve your solution

Output your answer concisely in the following format:

<thoughts>
[Your understanding of the task and feedback and how you plan to improve]
</thoughts>

<response>
[Your code implementation here]
</response>
"""




## タスクの記述
```
<user input>  
スタックを実装する 以下を使用:  
1. push(x)  
2. pop()  
3. getMin()  
全ての操作はO(1)であること.  
</user input>  
```

In [19]:
task = """
<user input>
Implement a Stack with:
1. push(x)
2. pop()
3. getMin()
All operations should be O(1).
</user input>
"""


In [22]:
result, chain_of_thought = loop(task, evaluator_prompt, generator_prompt)


=== GENERATION START ===
Thoughts:

The task requires implementing a Stack with constant time operations including finding minimum. 
To achieve O(1) for getMin(), we need to maintain a second stack that keeps track of minimums.
Each time we push, if the value is smaller than current min, we add it to minStack.
When we pop, if the popped value equals current min, we also pop from minStack.


Generated:

```python
class MinStack:
    def __init__(self):
        self.stack = []
        self.minStack = []
        
    def push(self, x: int) -> None:
        self.stack.append(x)
        if not self.minStack or x <= self.minStack[-1]:
            self.minStack.append(x)
            
    def pop(self) -> None:
        if not self.stack:
            return
        if self.stack[-1] == self.minStack[-1]:
            self.minStack.pop()
        self.stack.pop()
        
    def getMin(self) -> int:
        if not self.minStack:
            return None
        return self.minStack[-1]
```

=== G

In [27]:
from pprint import pprint

pprint(result)


('\n'
 '```python\n'
 'from typing import List\n'
 '\n'
 'class MinStack:\n'
 '    """A stack that supports push, pop, and getting minimum element in O(1) '
 'time."""\n'
 '    \n'
 '    def __init__(self):\n'
 '        """Initialize empty stack with two internal lists."""\n'
 '        self.stack: List[int] = []\n'
 '        self.minStack: List[int] = []\n'
 '        \n'
 '    def push(self, x: int) -> None:\n'
 '        """\n'
 '        Push element onto stack and update minimum stack.\n'
 '        Args:\n'
 '            x: Integer to push onto stack\n'
 '        Raises:\n'
 '            TypeError: If x is not an integer\n'
 '        """\n'
 '        if not isinstance(x, int):\n'
 '            raise TypeError("Input must be an integer")\n'
 '            \n'
 '        self.stack.append(x)\n'
 '        if not self.minStack or x <= self.minStack[-1]:\n'
 '            self.minStack.append(x)\n'
 '            \n'
 '    def pop(self) -> None:\n'
 '        """\n'
 '        Remove top element

In [29]:
pprint(chain_of_thought)

[{'result': '\n'
            '```python\n'
            'class MinStack:\n'
            '    def __init__(self):\n'
            '        self.stack = []\n'
            '        self.minStack = []\n'
            '        \n'
            '    def push(self, x: int) -> None:\n'
            '        self.stack.append(x)\n'
            '        if not self.minStack or x <= self.minStack[-1]:\n'
            '            self.minStack.append(x)\n'
            '            \n'
            '    def pop(self) -> None:\n'
            '        if not self.stack:\n'
            '            return\n'
            '        if self.stack[-1] == self.minStack[-1]:\n'
            '            self.minStack.pop()\n'
            '        self.stack.pop()\n'
            '        \n'
            '    def getMin(self) -> int:\n'
            '        if not self.minStack:\n'
            '            return None\n'
            '        return self.minStack[-1]\n'
            '```\n',
  'thoughts': '\n'
        